In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append('../')

from config.paths import RAW_DATA_PATH, PROCESSED_DATA_PATH
from utils.files_management import fix_csv_with_commas_in_text, load_netflix_data, load_multiple_netflix_files
from utils.data_processing import filter_sparse_users_and_movies, filter_valid_ratings, convert_columns_to_string
from utils.data_split import temporal_train_test_split

In [2]:
combined_data_1_path = RAW_DATA_PATH / "combined_data_1.txt"
combined_data_2_path = RAW_DATA_PATH / "combined_data_2.txt"
combined_data_3_path = RAW_DATA_PATH / "combined_data_3.txt"
combined_data_4_path = RAW_DATA_PATH / "combined_data_4.txt"

combined_data_path_list = [combined_data_1_path, combined_data_2_path, combined_data_3_path, combined_data_4_path]

concatenated_data = RAW_DATA_PATH / "data.parquet"

movie_titles_path = RAW_DATA_PATH / "movie_titles.csv"
movie_titles_fixed_path = RAW_DATA_PATH / "movie_titles_fixed.csv"


In [3]:
#df = load_multiple_netflix_files(
#    file_paths=combined_data_path_list,
#    save_path=concatenated_data,
#    verbose=False
#)

df = pd.read_parquet(concatenated_data)

In [4]:
#fix_csv_with_commas_in_text(movie_titles_path, movie_titles_fixed_path)
movie_titles = pd.read_csv(movie_titles_fixed_path, sep=';', encoding='latin1', header=None, names=['id', 'year', 'title'])

In [5]:
df.head()

,movie_id,customer_id,rating,date
0,1,1488844,3.0,2005-09-06
1,1,822109,5.0,2005-05-13
2,1,885013,4.0,2005-10-19
3,1,30878,4.0,2005-12-26
4,1,823519,3.0,2004-05-03


In [6]:
df.dtypes

movie_id                int64
customer_id             int64
rating                float64
date           datetime64[ns]
dtype: object

In [7]:
df.isna().mean()

movie_id       0.0
customer_id    0.0
rating         0.0
date           0.0
dtype: float64

# Preprocessing

In [8]:
df = convert_columns_to_string(df, ['customer_id', 'movie_id'])

In [9]:
# All the ratings are suposed to be in between 1-5
min_rating = 1
max_rating = 5
df = filter_valid_ratings(df, min_rating=min_rating, max_rating=max_rating)

In [ ]:
min_movie_ratings = 50
min_user_ratings = 10
df = filter_sparse_users_and_movies(df, min_movie_ratings=min_movie_ratings, min_user_ratings=min_user_ratings)

In [ ]:
processed_data_path = PROCESSED_DATA_PATH / "processed_data.parquet"
df.to_parquet(processed_data_path)